In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime as dt

In [2]:
files = os.listdir(path='data')

In [3]:
files

['friday.csv', 'monday.csv', 'thursday.csv', 'tuesday.csv', 'wednesday.csv']

In [4]:
def read_files(files, datapath='data/', delimiter = ';'):
        df_temp = []

        if type(files) == list:
            for i in files:
                df = pd.read_csv(datapath+i, delimiter=delimiter, parse_dates=True, index_col='timestamp')
                day_name = df.index.day_name()[0][:3]
                df['customer_no'] = df.customer_no.apply(lambda x: f'{day_name}_{x}')
                df_temp.append(df)
            df = pd.concat(df_temp)
        else:
            df = pd.read_csv(files, delimiter=delimiter, parse_dates=True, index_col='timestamp')
            day_name = df.index.day_name()[0][:3]
            df['customer_no'] = df.customer_no.apply(lambda x: f'{day_name}_{x}')

        return df

In [5]:
shop_data = read_files(files)

# Part 1: Data Exploration

In [6]:
shop_data = shop_data.sort_values(by='timestamp')

In [7]:
shop_data.sample(10)

,customer_no,location
timestamp,,
2019-09-02 14:04:00,Mon_639,checkout
2019-09-03 15:12:00,Tue_714,checkout
2019-09-05 08:07:00,Thu_112,dairy
2019-09-03 18:00:00,Tue_1003,spices
2019-09-02 12:40:00,Mon_486,checkout
2019-09-02 10:23:00,Mon_338,fruit
2019-09-06 11:07:00,Fri_395,fruit
2019-09-06 08:31:00,Fri_166,checkout
2019-09-03 14:22:00,Tue_632,dairy


##### Get list of all sections in the store

In [8]:
store_sections = list(shop_data.location.unique())
store_sections

['dairy', 'spices', 'fruit', 'checkout', 'drinks']

# Fill time steps

##### See customer follow throughout different sections

In [9]:
shop_data = shop_data.reset_index()

In [10]:
last_loc = shop_data.groupby('customer_no')['timestamp'].max()

In [11]:
shop_data.loc[shop_data.customer_no == 'Fri_1']

,timestamp,customer_no,location
19754,2019-09-06 07:00:00,Fri_1,dairy
19764,2019-09-06 07:04:00,Fri_1,spices
19768,2019-09-06 07:05:00,Fri_1,checkout


In [12]:
day_overtime = shop_data.groupby(['customer_no','location'])[['timestamp']].agg('min')\
.sort_values(by=['customer_no','timestamp'])

In [13]:
day_overtime

timestamp
customer_no location                    
Fri_1       dairy    2019-09-06 07:00:00
            spices   2019-09-06 07:04:00
            checkout 2019-09-06 07:05:00
Fri_10      fruit    2019-09-06 07:06:00
            checkout 2019-09-06 07:11:00
...                                  ...
Wed_998     dairy    2019-09-04 16:53:00
            fruit    2019-09-04 16:57:00
            checkout 2019-09-04 16:59:00
Wed_999     fruit    2019-09-04 16:53:00
            checkout 2019-09-04 16:54:00

[21671 rows x 1 columns]

In [14]:
customer_flow = pd.pivot_table(day_overtime, index='customer_no', values='timestamp', columns='location')\
.reindex(columns=['entrance','fruit', 'spices', 'dairy', 'drinks', 'checkout'])

In [15]:
customer_flow.loc[customer_flow.checkout.isnull()].head(1)

location,entrance,fruit,spices,dairy,drinks,checkout
customer_no,,,,,,
Fri_1494,NaN,2019-09-06 21:39:00,2019-09-06 21:49:00,2019-09-06 21:32:00,2019-09-06 21:33:00,NaT


# Fill in checkout time

In [16]:
def fill_timestamp(dataframe):
    if dataframe.checkout != dataframe.checkout:
        closing_time = dataframe.loc[dataframe.notnull()].max().round('H')
        return closing_time
    else:
        return dataframe.checkout
    
    if dataframe.checkout != dataframe.checkout:
        closing_time = dataframe.loc[dataframe.notnull()].max().round('H')
    return closing_time

In [17]:
customer_flow['checkout'] = customer_flow.apply(fill_timestamp, axis=1)

In [20]:
customers_in_sections = customer_flow.pivot(index=customer_flow.index.customer_no, columns=day_overtime.index.location, values='timestamp')
customers_in_sections

AttributeError: 'Index' object has no attribute 'customer_no'

In [19]:
time_test.index

NameError: name 'time_test' is not defined

In [ ]:
time_test = shop_data.set_index('timestamp')

In [ ]:
section_test = time_test[['customer_no', 'location']]

In [ ]:
#time_test.groupby(time_test.index)['location'].resample('1T').ffill()

In [ ]:
start_date = shop_data.timestamp.dt.date.min()
end_date = shop_data.timestamp.dt.date.max()

# Calculate the total number of customers in each section

In [ ]:
shop_data.groupby('location')['customer_no'].count().plot(kind='bar')
plt.xlabel('Store Sections')
plt.ylabel('Total Number of Customers')
plt.title('Visitation Numbers (by Section)')
plt.xticks(rotation=0)
plt.show()

locations = shop_data.groupby('location')['customer_no'].count().reset_index()
total_visits = locations.sum()

print(f'----Section Visitation Numbers----\n \
The following sections have had {total_visits[1]} vists between {start_date} and {end_date}:\n')
for idx, rows in locations.iterrows():
    print(f'{rows[0].title()}: {rows[1]}')


# Section counts over time

In [ ]:
day_overtime_counts = shop_data.groupby(['location', shop_data.timestamp.dt.hour])['customer_no'].count().reset_index()

In [ ]:
day_overtime_counts.head()

In [ ]:
sections_overtime = day_overtime_counts.pivot(index='timestamp', columns='location', values='customer_no')

In [ ]:
sections_overtime.plot()
sections_overtime

# Display the number of customers at checkout over time

In [ ]:
hour = shop_data.timestamp.dt.hour
checkout_overtime = shop_data.loc[shop_data.location == 'checkout'].groupby(hour)[['customer_no']].count()
checkout_overtime.head()

In [ ]:
checkout_overtime.plot(kind='line')
plt.xlabel(f'Trend from {start_date} to {end_date}')
plt.ylabel('Average Number of Customers')
plt.xticks(rotation=20)
plt.show()

day_totals = checkout_overtime.sum()

print(f'----Checkout Visitation Numbers----\n \
The checkout had a total of {int(day_totals)} vists between {start_date} and {end_date}:\n')
for idx, rows in checkout_overtime.iterrows():
    print(f'{idx}00: {int(rows[0])}')

In [ ]:
day_checkouts.plot(x='timestamp', y='customer_no', kind='bar')
plt.xlabel(f'Trend from {start_date} to {end_date}')
plt.ylabel('Average Number of Customers')
plt.xticks(rotation=20)
plt.show()

day_totals = day_checkouts['customer_no'].sum()

print(f'----Section Visitation Numbers----\n \
The following sections have had an average of {int(day_totals)} daily vists between {start_date} and {end_date}:\n')
for idx, rows in day_checkouts.iterrows():
    print(f'{rows[0]}: {int(rows[2])}')

# Calculate the time each customer spent in the market

In [ ]:
def total_time(dataframe):
    time_diff = dataframe[1] - dataframe[0]
    
    total_time = time_diff.seconds / 60
    
    return time_diff.seconds

In [ ]:
shop_data

In [ ]:
shop_data['day'] = shop_data.index.day_name()

In [ ]:
shop_data.reset_index(inplace=True)

In [ ]:
customers = shop_data.groupby(['customer_no'])['timestamp'].agg(['min', 'max'])

In [ ]:
customers

In [ ]:
customers['total_shop_time (seconds)'] = customers[['min','max']].apply(total_time, axis=1)

In [ ]:
total_avg_time_spent = round(customers['total_shop_time (seconds)'].mean(),1)

In [ ]:
daily_avg_time_spent = customers.groupby('day')[['total_shop_time (seconds)']].mean().reset_index()
daily_avg_time_spent

In [ ]:
daily_avg_time_spent.plot(x='day', y='total_shop_time (seconds)', kind='bar')
plt.xlabel(f'Trend from {start_date} to {end_date}')
plt.ylabel('Average Shopping Time (in seconds)')
plt.title('Average Time Spent Shopping')
plt.xticks(rotation=20)
plt.show()

print(f'----Section Visitation Numbers----\n \
Customers spend on average {total_avg_time_spent} seconds shopping on the days between {start_date} and {end_date}:\n')
for idx, rows in daily_avg_time_spent.iterrows():
    print(f'{rows[0]}: {round(rows[1], 2)} seconds spent in the store')

In [ ]:
t = shop_data.groupby('day')